In [ ]:
%matplotlib inline

from hydromt.log import setuplog
from hydromt_sfincs import SfincsModel, utils
from hydromt.config import configread
import geopandas as gpd

import subprocess

import hydromt
from hydromt import DataCatalog
import datetime
import xarray as xr
import sys, os
from os.path import join
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, date
# sys.path.append('./')
# import HYDmodfunctions as HM

Hydraulic evaluation: Influence of both rivers in the flood patterns

In [ ]:
#Figure: Inundation map
def Fig(rp1, rp2):
    tittle = f'S{rp1}_S{rp2}'

    folder = r"..\confluence_run_S{}_S{}".format(rp1,rp2)
    mod = SfincsModel(folder, mode='r')

    fig, ax = mod.plot_basemap(
        fn_out=None,
        variable=None,
        bmap="sat",
        geoms=["src", "obs"],
        plot_bounds=False,
        figsize=(11, 7),
    )

    hmin = 0.2  # minimum flood depth [m] to plot

    da_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

    da_hmax_fld = da_hmax.where(da_hmax > hmin)
    # da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
    da_hmax.attrs.update(long_name="flood depth", unit="m")

    # plot overland flooding based on gswo mask and mimum flood depth
    cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}
    cax_fld = da_hmax_fld.plot(
        ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.viridis, cbar_kwargs=cbar_kwargs
    )

    ax.set_title(f"SFINCS maximum water depth | run {tittle}")
    # cax_fld.savefig(folder + r'hmax.png')
    return

In [ ]:
plt.rcParams['font.size'] = '13'
plt.rcParams['xtick.labelsize'] = '10'
plt.rcParams['savefig.facecolor']='white'

In [ ]:
S1 = np.array([  723.95,  3403.62,  6083.29,  8762.96, 11442.62])
S2 = np.array([  27.14, 1119.63, 2212.12, 3304.61, 4397.12])
S1_n = ['S0', 'S1', 'S2', 'S3', 'S4']   #Colum
S2_n = ['S0', 'S1', 'S2', 'S3', 'S4']   #INDEX

For calculating the differences with respect the "worst" scenario S5_S5

In [ ]:
rp1 = 4
rp2 = 4
Fig(rp1, rp2)

In [ ]:
def Diff(rp1, rp2):
    rp1_b = 4
    rp2_b = 4
    tittle = f'Difference between run S{rp1_b}_S{rp2_b} and run S{rp1}_S{rp2}'

    #Benchmarck
    folder = r"..\confluence_run_S{}_S{}".format(rp1_b,rp2_b)
    mod = SfincsModel(folder, mode='r')

    fig, ax = mod.plot_basemap(
        fn_out=None,
        variable=None,
        bmap="sat",
        geoms=["src", "obs"],
        plot_bounds=False,
        figsize=(11, 7),
    )

    hmin = 0.2  # minimum flood depth [m] to plot

    bm_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

    bm_hmax_fld = bm_hmax.where(bm_hmax > hmin)
    # da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
    bm_hmax.attrs.update(long_name="flood depth", unit="m")

    # To compare
    folder = r"..\confluence_run_S{}_S{}".format(rp1,rp2)
    mod = SfincsModel(folder, mode='r')

    # fig, ax = mod.plot_basemap(
    #     fn_out=None,
    #     variable=None,
    #     bmap="sat",
    #     geoms=["src", "obs"],
    #     plot_bounds=False,
    #     figsize=(11, 7),
    # )

    hmin = 0.2  # minimum flood depth [m] to plot

    da_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

    da_hmax_fld = da_hmax.where(da_hmax > hmin)
    # da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
    da_hmax.attrs.update(long_name="flood depth", unit="m")

    #COMPARISON
    # hdif = bm_hmax_fld - da_hmax_fld
    hdif = bm_hmax_fld.fillna(0) - da_hmax_fld.fillna(0)

    # plot overland flooding based on gswo mask and mimum flood depth
    cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}
    # cax_fld = da_hmax_fld.plot(
    #     ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.viridis, cbar_kwargs=cbar_kwargs
    # )
    cax_fld = hdif.plot(
        ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.Reds, cbar_kwargs=cbar_kwargs
    )

    ax.set_title(f"SFINCS maximum water depth \n {tittle} \n")
    # cax_fld.savefig(folder + r'hmax.png')
    plt.savefig(join(mod.root, 'figs', 'hmax_DiffS4S4.png'), dpi=225, bbox_inches="tight")
    return

In [ ]:
for s1 in range(len(S1_n)): #(len(S1_n))   (0, 1)
    for s2 in range(len(S2_n)): #(len(S2_n))
        rp1 = s1 #from S1 [0, 1, 2, 3, 4]
        rp2 = s2 #from S2 [0, 1, 2, 3, 4]
        TSpath = r'\TS_S%d' %rp1 + r'_S%d.csv' %rp2 #  according to Q1_Q2
        fn = r'P:\11206883-006-dar-cloud-computing\GRADE_export\HYDa\RQ3\data\TSgrid' + TSpath
        print(f'Starting {TSpath} at {str(datetime.now())}')
        Diff(rp1, rp2)
        print(f'Finishing {TSpath} at {str(datetime.now())}')

In [ ]:
def Diff2(rp1_b, rp2_b, rp1, rp2):
    tittle = f'Difference between run S{rp1_b}_S{rp2_b} and run S{rp1}_S{rp2}'
    fig_tittle = f'hmax_DiffS{rp1_b}_S{rp2_b}.png'

    #Benchmarck
    folder = r"..\confluence_run_S{}_S{}".format(rp1_b,rp2_b)
    mod = SfincsModel(folder, mode='r')

    fig, ax = mod.plot_basemap(
        fn_out=None,
        variable=None,
        bmap="sat",
        geoms=["src", "obs"],
        plot_bounds=False,
        figsize=(11, 7),
    )

    hmin = 0.2  # minimum flood depth [m] to plot

    bm_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

    bm_hmax_fld = bm_hmax.where(bm_hmax > hmin)
    # da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
    bm_hmax.attrs.update(long_name="flood depth", unit="m")

    # To compare
    folder = r"..\confluence_run_S{}_S{}".format(rp1,rp2)
    mod = SfincsModel(folder, mode='r')

    # fig, ax = mod.plot_basemap(
    #     fn_out=None,
    #     variable=None,
    #     bmap="sat",
    #     geoms=["src", "obs"],
    #     plot_bounds=False,
    #     figsize=(11, 7),
    # )

    hmin = 0.2  # minimum flood depth [m] to plot

    da_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

    da_hmax_fld = da_hmax.where(da_hmax > hmin)
    # da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
    da_hmax.attrs.update(long_name="flood depth", unit="m")

    #COMPARISON
    # hdif = bm_hmax_fld - da_hmax_fld
    hdif = bm_hmax_fld.fillna(0) - da_hmax_fld.fillna(0)

    # plot overland flooding based on gswo mask and mimum flood depth
    cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}
    # cax_fld = da_hmax_fld.plot(
    #     ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.viridis, cbar_kwargs=cbar_kwargs
    # )
    cax_fld = hdif.plot(
        ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.Reds, cbar_kwargs=cbar_kwargs
    )

    ax.set_title(f"SFINCS maximum water depth \n {tittle} \n")
    # cax_fld.savefig(folder + r'hmax.png')
    # plt.savefig(join(mod.root, 'figs', fig_tittle), dpi=225, bbox_inches="tight")
    return

In [ ]:
rp1_b = 4
rp2_b = 0
rp1 = 0
rp2 = 0

tittle = f'Difference between run S{rp1_b}_S{rp2_b} and run S{rp1}_S{rp2}'
fig_tittle = f'hmax_DiffS{rp1_b}_S{rp2_b}.png'

#Benchmarck
folder = r"..\confluence_run_S{}_S{}".format(rp1_b,rp2_b)
mod = SfincsModel(folder, mode='r')

fig, ax = mod.plot_basemap(
    fn_out=None,
    variable=None,
    bmap="sat",
    geoms=["src", "obs"],
    plot_bounds=False,
    figsize=(11, 7),
)

hmin = 0.2  # minimum flood depth [m] to plot

bm_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

bm_hmax_fld = bm_hmax.where(bm_hmax > hmin)
# da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
bm_hmax.attrs.update(long_name="flood depth", unit="m")

# To compare
folder = r"..\confluence_run_S{}_S{}".format(rp1,rp2)
mod = SfincsModel(folder, mode='r')

# fig, ax = mod.plot_basemap(
#     fn_out=None,
#     variable=None,
#     bmap="sat",
#     geoms=["src", "obs"],
#     plot_bounds=False,
#     figsize=(11, 7),
# )

hmin = 0.2  # minimum flood depth [m] to plot

da_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

da_hmax_fld = da_hmax.where(da_hmax > hmin)
# da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
da_hmax.attrs.update(long_name="flood depth", unit="m")

#COMPARISON
# hdif = bm_hmax_fld - da_hmax_fld
hdif = bm_hmax_fld.fillna(0) - da_hmax_fld.fillna(0)

# plot overland flooding based on gswo mask and mimum flood depth
cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}
# cax_fld = da_hmax_fld.plot(
#     ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.viridis, cbar_kwargs=cbar_kwargs
# )
cax_fld = hdif.plot(
    ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.Reds, cbar_kwargs=cbar_kwargs
)

ax.set_title(f"SFINCS maximum water depth \n {tittle} \n")
plt.savefig(join(mod.root, 'figs', fig_tittle), dpi=225, bbox_inches="tight")

In [ ]:
rp1_b = 4
rp2_b = 1
rp1 = 0
rp2 = 1

tittle = f'Difference between run S{rp1_b}_S{rp2_b} and run S{rp1}_S{rp2}'
fig_tittle = f'hmax_DiffS{rp1_b}_S{rp2_b}.png'

#Benchmarck
folder = r"..\confluence_run_S{}_S{}".format(rp1_b,rp2_b)
mod = SfincsModel(folder, mode='r')

fig, ax = mod.plot_basemap(
    fn_out=None,
    variable=None,
    bmap="sat",
    geoms=["src", "obs"],
    plot_bounds=False,
    figsize=(11, 7),
)

hmin = 0.2  # minimum flood depth [m] to plot

bm_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

bm_hmax_fld1 = bm_hmax.where(bm_hmax > hmin)
# da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
bm_hmax.attrs.update(long_name="flood depth", unit="m")

# To compare
folder = r"..\confluence_run_S{}_S{}".format(rp1,rp2)
mod = SfincsModel(folder, mode='r')

# fig, ax = mod.plot_basemap(
#     fn_out=None,
#     variable=None,
#     bmap="sat",
#     geoms=["src", "obs"],
#     plot_bounds=False,
#     figsize=(11, 7),
# )

hmin = 0.2  # minimum flood depth [m] to plot

da_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

da_hmax_fld1 = da_hmax.where(da_hmax > hmin)
# da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
da_hmax.attrs.update(long_name="flood depth", unit="m")

#COMPARISON
# hdif = bm_hmax_fld - da_hmax_fld
hdif = bm_hmax_fld1.fillna(0) - da_hmax_fld1.fillna(0)

# plot overland flooding based on gswo mask and mimum flood depth
cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}
# cax_fld = da_hmax_fld.plot(
#     ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.viridis, cbar_kwargs=cbar_kwargs
# )
cax_fld = hdif.plot(
    ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.Reds, cbar_kwargs=cbar_kwargs
)

ax.set_title(f"SFINCS maximum water depth \n {tittle} \n")
plt.savefig(join(mod.root, 'figs', fig_tittle), dpi=225, bbox_inches="tight")

In [ ]:
rp1_b = 4
rp2_b = 2
rp1 = 0
rp2 = 2

tittle = f'Difference between run S{rp1_b}_S{rp2_b} and run S{rp1}_S{rp2}'
fig_tittle = f'hmax_DiffS{rp1_b}_S{rp2_b}.png'

#Benchmarck
folder = r"..\confluence_run_S{}_S{}".format(rp1_b,rp2_b)
mod = SfincsModel(folder, mode='r')

fig, ax = mod.plot_basemap(
    fn_out=None,
    variable=None,
    bmap="sat",
    geoms=["src", "obs"],
    plot_bounds=False,
    figsize=(11, 7),
)

hmin = 0.2  # minimum flood depth [m] to plot

bm_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

bm_hmax_fld2 = bm_hmax.where(bm_hmax > hmin)
# da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
bm_hmax.attrs.update(long_name="flood depth", unit="m")

# To compare
folder = r"..\confluence_run_S{}_S{}".format(rp1,rp2)
mod = SfincsModel(folder, mode='r')

# fig, ax = mod.plot_basemap(
#     fn_out=None,
#     variable=None,
#     bmap="sat",
#     geoms=["src", "obs"],
#     plot_bounds=False,
#     figsize=(11, 7),
# )

hmin = 0.2  # minimum flood depth [m] to plot

da_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

da_hmax_fld2 = da_hmax.where(da_hmax > hmin)
# da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
da_hmax.attrs.update(long_name="flood depth", unit="m")

#COMPARISON
# hdif = bm_hmax_fld - da_hmax_fld
hdif = bm_hmax_fld2.fillna(0) - da_hmax_fld2.fillna(0)

# plot overland flooding based on gswo mask and mimum flood depth
cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}
# cax_fld = da_hmax_fld.plot(
#     ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.viridis, cbar_kwargs=cbar_kwargs
# )
cax_fld = hdif.plot(
    ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.Reds, cbar_kwargs=cbar_kwargs
)

ax.set_title(f"SFINCS maximum water depth \n {tittle} \n")
plt.savefig(join(mod.root, 'figs', fig_tittle), dpi=225, bbox_inches="tight")

In [ ]:
rp1_b = 4
rp2_b = 3
rp1 = 0
rp2 = 3

tittle = f'Difference between run S{rp1_b}_S{rp2_b} and run S{rp1}_S{rp2}'
fig_tittle = f'hmax_DiffS{rp1_b}_S{rp2_b}.png'

#Benchmarck
folder = r"..\confluence_run_S{}_S{}".format(rp1_b,rp2_b)
mod = SfincsModel(folder, mode='r')

fig, ax = mod.plot_basemap(
    fn_out=None,
    variable=None,
    bmap="sat",
    geoms=["src", "obs"],
    plot_bounds=False,
    figsize=(11, 7),
)

hmin = 0.2  # minimum flood depth [m] to plot

bm_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

bm_hmax_fld3 = bm_hmax.where(bm_hmax > hmin)
# da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
bm_hmax.attrs.update(long_name="flood depth", unit="m")

# To compare
folder = r"..\confluence_run_S{}_S{}".format(rp1,rp2)
mod = SfincsModel(folder, mode='r')

# fig, ax = mod.plot_basemap(
#     fn_out=None,
#     variable=None,
#     bmap="sat",
#     geoms=["src", "obs"],
#     plot_bounds=False,
#     figsize=(11, 7),
# )

hmin = 0.2  # minimum flood depth [m] to plot

da_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

da_hmax_fld3 = da_hmax.where(da_hmax > hmin)
# da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
da_hmax.attrs.update(long_name="flood depth", unit="m")

#COMPARISON
# hdif = bm_hmax_fld - da_hmax_fld
hdif = bm_hmax_fld3.fillna(0) - da_hmax_fld3.fillna(0)

# plot overland flooding based on gswo mask and mimum flood depth
cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}
# cax_fld = da_hmax_fld.plot(
#     ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.viridis, cbar_kwargs=cbar_kwargs
# )
cax_fld = hdif.plot(
    ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.Reds, cbar_kwargs=cbar_kwargs
)

ax.set_title(f"SFINCS maximum water depth \n {tittle} \n")
plt.savefig(join(mod.root, 'figs', fig_tittle), dpi=225, bbox_inches="tight")

In [ ]:
rp1_b = 4
rp2_b = 4
rp1 = 0
rp2 = 4

tittle = f'Difference between run S{rp1_b}_S{rp2_b} and run S{rp1}_S{rp2}'
fig_tittle = f'hmax_DiffS{rp1_b}_S{rp2_b}.png'

#Benchmarck
folder = r"..\confluence_run_S{}_S{}".format(rp1_b,rp2_b)
mod = SfincsModel(folder, mode='r')

fig, ax = mod.plot_basemap(
    fn_out=None,
    variable=None,
    bmap="sat",
    geoms=["src", "obs"],
    plot_bounds=False,
    figsize=(11, 7),
)

hmin = 0.2  # minimum flood depth [m] to plot

bm_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

bm_hmax_fld10 = bm_hmax.where(bm_hmax > hmin)
# da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
bm_hmax.attrs.update(long_name="flood depth", unit="m")

# To compare
folder = r"..\confluence_run_S{}_S{}".format(rp1,rp2)
mod = SfincsModel(folder, mode='r')

# fig, ax = mod.plot_basemap(
#     fn_out=None,
#     variable=None,
#     bmap="sat",
#     geoms=["src", "obs"],
#     plot_bounds=False,
#     figsize=(11, 7),
# )

hmin = 0.2  # minimum flood depth [m] to plot

da_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

da_hmax_fld10 = da_hmax.where(da_hmax > hmin)
# da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
da_hmax.attrs.update(long_name="flood depth", unit="m")

#COMPARISON
# hdif = bm_hmax_fld - da_hmax_fld
hdif = bm_hmax_fld10.fillna(0) - da_hmax_fld10.fillna(0)

# plot overland flooding based on gswo mask and mimum flood depth
cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}
# cax_fld = da_hmax_fld.plot(
#     ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.viridis, cbar_kwargs=cbar_kwargs
# )
cax_fld = hdif.plot(
    ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.Reds, cbar_kwargs=cbar_kwargs
)

ax.set_title(f"SFINCS maximum water depth \n {tittle} \n")
plt.savefig(join(mod.root, 'figs', fig_tittle), dpi=225, bbox_inches="tight")

Main differences

In [ ]:
rp1_b = 0
rp2_b = 4
rp1 = 0
rp2 = 0

tittle = f'Difference between run S{rp1_b}_S{rp2_b} and run S{rp1}_S{rp2}'
fig_tittle = f'hmax_DiffS{rp1_b}_S{rp2_b}.png'

#Benchmarck
folder = r"..\confluence_run_S{}_S{}".format(rp1_b,rp2_b)
mod = SfincsModel(folder, mode='r')

fig, ax = mod.plot_basemap(
    fn_out=None,
    variable=None,
    bmap="sat",
    geoms=["src", "obs"],
    plot_bounds=False,
    figsize=(11, 7),
)

hmin = 0.2  # minimum flood depth [m] to plot

bm_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

bm_hmax_fld5 = bm_hmax.where(bm_hmax > hmin)
# da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
bm_hmax.attrs.update(long_name="flood depth", unit="m")

# To compare
folder = r"..\confluence_run_S{}_S{}".format(rp1,rp2)
mod = SfincsModel(folder, mode='r')

# fig, ax = mod.plot_basemap(
#     fn_out=None,
#     variable=None,
#     bmap="sat",
#     geoms=["src", "obs"],
#     plot_bounds=False,
#     figsize=(11, 7),
# )

hmin = 0.2  # minimum flood depth [m] to plot

da_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

da_hmax_fld5 = da_hmax.where(da_hmax > hmin)
# da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
da_hmax.attrs.update(long_name="flood depth", unit="m")

#COMPARISON
# hdif = bm_hmax_fld - da_hmax_fld
hdif = bm_hmax_fld5.fillna(0) - da_hmax_fld5.fillna(0)

# plot overland flooding based on gswo mask and mimum flood depth
cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}
# cax_fld = da_hmax_fld.plot(
#     ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.viridis, cbar_kwargs=cbar_kwargs
# )
cax_fld = hdif.plot(
    ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.Reds, cbar_kwargs=cbar_kwargs
)

ax.set_title(f"SFINCS maximum water depth \n {tittle} \n")
plt.savefig(join(mod.root, 'figs', fig_tittle), dpi=225, bbox_inches="tight")

In [ ]:
rp1_b = 1
rp2_b = 4
rp1 = 1
rp2 = 0

tittle = f'Difference between run S{rp1_b}_S{rp2_b} and run S{rp1}_S{rp2}'
fig_tittle = f'hmax_DiffS{rp1_b}_S{rp2_b}.png'

#Benchmarck
folder = r"..\confluence_run_S{}_S{}".format(rp1_b,rp2_b)
mod = SfincsModel(folder, mode='r')

fig, ax = mod.plot_basemap(
    fn_out=None,
    variable=None,
    bmap="sat",
    geoms=["src", "obs"],
    plot_bounds=False,
    figsize=(11, 7),
)

hmin = 0.2  # minimum flood depth [m] to plot

bm_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

bm_hmax_fld6 = bm_hmax.where(bm_hmax > hmin)
# da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
bm_hmax.attrs.update(long_name="flood depth", unit="m")

# To compare
folder = r"..\confluence_run_S{}_S{}".format(rp1,rp2)
mod = SfincsModel(folder, mode='r')

# fig, ax = mod.plot_basemap(
#     fn_out=None,
#     variable=None,
#     bmap="sat",
#     geoms=["src", "obs"],
#     plot_bounds=False,
#     figsize=(11, 7),
# )

hmin = 0.2  # minimum flood depth [m] to plot

da_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

da_hmax_fld6 = da_hmax.where(da_hmax > hmin)
# da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
da_hmax.attrs.update(long_name="flood depth", unit="m")

#COMPARISON
# hdif = bm_hmax_fld - da_hmax_fld
hdif = bm_hmax_fld6.fillna(0) - da_hmax_fld6.fillna(0)

# plot overland flooding based on gswo mask and mimum flood depth
cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}
# cax_fld = da_hmax_fld.plot(
#     ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.viridis, cbar_kwargs=cbar_kwargs
# )
cax_fld = hdif.plot(
    ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.Reds, cbar_kwargs=cbar_kwargs
)

ax.set_title(f"SFINCS maximum water depth \n {tittle} \n")
plt.savefig(join(mod.root, 'figs', fig_tittle), dpi=225, bbox_inches="tight")

In [ ]:
rp1_b = 2
rp2_b = 4
rp1 = 2
rp2 = 0

tittle = f'Difference between run S{rp1_b}_S{rp2_b} and run S{rp1}_S{rp2}'
fig_tittle = f'hmax_DiffS{rp1_b}_S{rp2_b}.png'

#Benchmarck
folder = r"..\confluence_run_S{}_S{}".format(rp1_b,rp2_b)
mod = SfincsModel(folder, mode='r')

fig, ax = mod.plot_basemap(
    fn_out=None,
    variable=None,
    bmap="sat",
    geoms=["src", "obs"],
    plot_bounds=False,
    figsize=(11, 7),
)

hmin = 0.2  # minimum flood depth [m] to plot

bm_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

bm_hmax_fld7 = bm_hmax.where(bm_hmax > hmin)
# da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
bm_hmax.attrs.update(long_name="flood depth", unit="m")

# To compare
folder = r"..\confluence_run_S{}_S{}".format(rp1,rp2)
mod = SfincsModel(folder, mode='r')

# fig, ax = mod.plot_basemap(
#     fn_out=None,
#     variable=None,
#     bmap="sat",
#     geoms=["src", "obs"],
#     plot_bounds=False,
#     figsize=(11, 7),
# )

hmin = 0.2  # minimum flood depth [m] to plot

da_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

da_hmax_fld7 = da_hmax.where(da_hmax > hmin)
# da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
da_hmax.attrs.update(long_name="flood depth", unit="m")

#COMPARISON
# hdif = bm_hmax_fld - da_hmax_fld
hdif = bm_hmax_fld7.fillna(0) - da_hmax_fld7.fillna(0)

# plot overland flooding based on gswo mask and mimum flood depth
cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}
# cax_fld = da_hmax_fld.plot(
#     ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.viridis, cbar_kwargs=cbar_kwargs
# )
cax_fld = hdif.plot(
    ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.Reds, cbar_kwargs=cbar_kwargs
)

ax.set_title(f"SFINCS maximum water depth \n {tittle} \n")
plt.savefig(join(mod.root, 'figs', fig_tittle), dpi=225, bbox_inches="tight")

In [ ]:
rp1_b = 3
rp2_b = 4
rp1 = 3
rp2 = 0

tittle = f'Difference between run S{rp1_b}_S{rp2_b} and run S{rp1}_S{rp2}'
fig_tittle = f'hmax_DiffS{rp1_b}_S{rp2_b}.png'

#Benchmarck
folder = r"..\confluence_run_S{}_S{}".format(rp1_b,rp2_b)
mod = SfincsModel(folder, mode='r')

fig, ax = mod.plot_basemap(
    fn_out=None,
    variable=None,
    bmap="sat",
    geoms=["src", "obs"],
    plot_bounds=False,
    figsize=(11, 7),
)

hmin = 0.2  # minimum flood depth [m] to plot

bm_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

bm_hmax_fld8 = bm_hmax.where(bm_hmax > hmin)
# da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
bm_hmax.attrs.update(long_name="flood depth", unit="m")

# To compare
folder = r"..\confluence_run_S{}_S{}".format(rp1,rp2)
mod = SfincsModel(folder, mode='r')

# fig, ax = mod.plot_basemap(
#     fn_out=None,
#     variable=None,
#     bmap="sat",
#     geoms=["src", "obs"],
#     plot_bounds=False,
#     figsize=(11, 7),
# )

hmin = 0.2  # minimum flood depth [m] to plot

da_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

da_hmax_fld8 = da_hmax.where(da_hmax > hmin)
# da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
da_hmax.attrs.update(long_name="flood depth", unit="m")

#COMPARISON
# hdif = bm_hmax_fld - da_hmax_fld
hdif = bm_hmax_fld8.fillna(0) - da_hmax_fld8.fillna(0)

# plot overland flooding based on gswo mask and mimum flood depth
cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}
# cax_fld = da_hmax_fld.plot(
#     ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.viridis, cbar_kwargs=cbar_kwargs
# )
cax_fld = hdif.plot(
    ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.Reds, cbar_kwargs=cbar_kwargs
)

ax.set_title(f"SFINCS maximum water depth \n {tittle} \n")
plt.savefig(join(mod.root, 'figs', fig_tittle), dpi=225, bbox_inches="tight")

In [ ]:
rp1_b = 4
rp2_b = 4
rp1 = 4
rp2 = 0

tittle = f'Difference between run S{rp1_b}_S{rp2_b} and run S{rp1}_S{rp2}'
fig_tittle = f'hmax_DiffS{rp1_b}_S{rp2_b}.png'

#Benchmarck
folder = r"..\confluence_run_S{}_S{}".format(rp1_b,rp2_b)
mod = SfincsModel(folder, mode='r')

fig, ax = mod.plot_basemap(
    fn_out=None,
    variable=None,
    bmap="sat",
    geoms=["src", "obs"],
    plot_bounds=False,
    figsize=(11, 7),
)

hmin = 0.2  # minimum flood depth [m] to plot

bm_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

bm_hmax_fld9 = bm_hmax.where(bm_hmax > hmin)
# da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
bm_hmax.attrs.update(long_name="flood depth", unit="m")

# To compare
folder = r"..\confluence_run_S{}_S{}".format(rp1,rp2)
mod = SfincsModel(folder, mode='r')

# fig, ax = mod.plot_basemap(
#     fn_out=None,
#     variable=None,
#     bmap="sat",
#     geoms=["src", "obs"],
#     plot_bounds=False,
#     figsize=(11, 7),
# )

hmin = 0.2  # minimum flood depth [m] to plot

da_hmax = mod.results["hmax"]  # hmax is computed from zsmax - zb

da_hmax_fld9 = da_hmax.where(da_hmax > hmin)
# da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)
da_hmax.attrs.update(long_name="flood depth", unit="m")

#COMPARISON
# hdif = bm_hmax_fld - da_hmax_fld
hdif = bm_hmax_fld9.fillna(0) - da_hmax_fld9.fillna(0)

# plot overland flooding based on gswo mask and mimum flood depth
cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}
# cax_fld = da_hmax_fld.plot(
#     ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.viridis, cbar_kwargs=cbar_kwargs
# )
cax_fld = hdif.plot(
    ax=ax, vmin=0, vmax=3.0, cmap=plt.cm.Reds, cbar_kwargs=cbar_kwargs
)

ax.set_title(f"SFINCS maximum water depth \n {tittle} \n")
plt.savefig(join(mod.root, 'figs', fig_tittle), dpi=225, bbox_inches="tight")